In [1]:
import pickle
import pandas as pd
from datetime import datetime, date, time, timedelta
from collections import defaultdict, Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import textrank
import os
import sys
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from rake_nltk import Rake
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict
from math import log, floor
import numpy as np
import textrank
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, recall_score, precision_score, precision_recall_fscore_support
from sklearn.metrics import matthews_corrcoef
import pickle
import os
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
stop_words = stopwords.words("english")
stop_words.extend(["chris", "satterfield"])

In [2]:
total_counter = None

def tfidf_cosine_similarity(task, description):
    global total_counter
    c = Counter(description)
    for key in c:
        if key in total_counter:
            c[key] = c[key]/total_counter[key]
            
    v1 = []
    v2 = []
    for key in total_counter:
        if key in task:
            v1.append(task[key])
        else:
            v1.append(0)
            
        if key in c:
            v2.append(c[key])
        else:
            v2.append(0)
            
            
    return sim(np.array(v1), np.array(v2))
    
    

def rake(snapshot):
    r = Rake()
    r.extract_keywords_from_text(snapshot)
    keyphrases = r.get_ranked_phrases()
    tokens = []
    [tokens.extend(x.split(" ")) for x in keyphrases]
    return tokens

def valid(dateobj, start, end):
    return dateobj >= start and dateobj < end        

def tfidf(task_counters, weighted_df):
    global total_counter
    idf_counters = []
    for counter in task_counters:
        n = Counter()
        for key in counter:
            n[key] = 1
        idf_counters.append(n)
    total_counter = sum(idf_counters, Counter())
    
    weighted_counters = []
    for counter in task_counters:
        occurances = []
        for key in counter.keys():
            occurances.append(total_counter[key])
        
        mean = np.mean(occurances)
        
        weighted = {}
        for key in counter.keys():
            if(weighted_df):
                weighted[key] = counter[key]/((total_counter[key])  * (1 + abs(mean - total_counter[key])))
            else:
                weighted[key] = counter[key] /(total_counter[key])
        weighted_counters.append(Counter(weighted))
    return weighted_counters


In [3]:
cache = {}

In [11]:
def shuffleDict(dictionary):
    keys = list(dictionary.keys())
    random.shuffle(keys)
    shuffled = {}
    for key in keys:
        shuffled[key] = dictionary[key]
    
    return shuffled

def equals(prediction, expected):
    if(prediction == expected):
        return 1
    else:
        return 0

def get_prediction(scores):
    scores = shuffleDict(scores)
    pred = max(scores, key=scores.get)
    #choices = list(range(1,len(scores) + 1))
    #choices.append(-1)
    #return np.random.choice(choices)
    #if(scores[pred] < 0.05):
    #    return -1
    #else:
    return pred

def normalize_score(scores):
    norm = np.linalg.norm(list(scores.values()), ord=1)
    for score in scores:
        if(norm != 0):
            scores[score] = scores[score] / norm
    return scores

class WindowTitleTaskExtractor:

    def get_tasks_for_participant(self, path_to_data, participant, tokenizer, weighted=False, without_emails=True, using_tfidf=False, ungrouped=False, widf=False):
        
        appdata = pd.read_csv(f"{path_to_data}/{participant}/appdata_fixed.csv")
        df = pd.read_excel(f"{path_to_data}/{participant}/taskswitches_annotated.xlsx")
        offset = df[df["task"] == "offset"]["end"].iloc[0]
        df = df[df["task"] != "offset"]
        task_words_ungrouped = []
        task_order = []

        studyStartTime = appdata["StartTime"][0] - (datetime.combine(date.min, offset) - datetime.min).total_seconds()

        for _,row in df.iterrows():
            startDelta = datetime.combine(date.min, row["start"]) - datetime.min
            endDelta = datetime.combine(date.min, row["end"]) - datetime.min
            start = studyStartTime + startDelta.total_seconds()
            end = studyStartTime + endDelta.total_seconds()

            titles = appdata[((appdata["StartTime"] >= start) & (appdata["StartTime"] < end)) | ((appdata["EndTime"] >= start) & (appdata["EndTime"] < end))]
            titles = list(titles["WindowTitle"])
            titles = [x.lower() for x in titles]

            if(without_emails):
                titles = [x for x in titles if not "gmail" in x]
            
            words = []
            for title in titles:
                tokens = word_tokenize(title)
                snapshot_words = [x for x in tokens if len(x) > 2]
                snapshot_words = [stemmer.stem(x) for x in snapshot_words if not x in stop_words]
                words.extend(snapshot_words)

            if(len(words) > 0):
                task_words_ungrouped.append(words)
                task_order.append(row["task"])
        
        task_counters = [Counter(x) for x in task_words_ungrouped]
        if(not ungrouped):
            counters = defaultdict(Counter)
            for counter, task in zip(task_counters, task_order):
                counters[task] += counter
            task_order, task_counters = zip(*counters.items())
            
        if(using_tfidf):
            task_counters = tfidf(task_counters, weighted)

        for task in task_counters:
            norm = np.linalg.norm(list(task.values()))
            for word in task:
                task[word] = task[word]/norm

        return task_counters, task_order

            
            
class ScreenshotTaskExtractor(object):

    def isProbablyEmail(self, task):
        excludedWords = ['compose', 'gmail', 'inbox', 'google', 'starred', 'sent','mail','drafts','more','terms','privacy','program','policies']
        i = 0
        for word in excludedWords:
            if word in task:
                i += 1
        return i > 4

    def get_tasks_for_participant(self, path_to_data, participant, tokenizer, weighted=False, without_emails=True, ungrouped=False, using_tfidf=False):
        with open(f"{path_to_data}/{participant}/fulltext.pkl", "rb") as f:
            snapshotsWithDates = pickle.load(f)
        
        df = pd.read_excel(f"{path_to_data}/{participant}/taskswitches_annotated.xlsx")
        offset = df[df["task"] == "offset"]["end"].iloc[0]
        startDelta = datetime.combine(date.min, offset) - datetime.min 
        df = df[df["task"] != "offset"]
        task_words_ungrouped = []
        task_order = []
        

        studyStartTime = snapshotsWithDates[0][0] - (datetime.combine(date.min, offset) - datetime.min)
        start = studyStartTime

        for _,row in df.iterrows():
            startDelta = datetime.combine(date.min, row["start"]) - datetime.min 
            endDelta = datetime.combine(date.min, row["end"]) - datetime.min
            start = studyStartTime + startDelta
            taskEnd = studyStartTime + endDelta
            while(start < taskEnd):
                end = start + timedelta(seconds=60)
                snapshotsInTask = [x[1].lower() for x in snapshotsWithDates if valid(x[0], start, end)]
                start = start + timedelta(seconds=60)

                if(without_emails):
                    snapshotsInTask = [x for x in snapshotsInTask if not self.isProbablyEmail(x)]

                words = []
                for snapshot in snapshotsInTask:
                    tokens = tokenizer(snapshot)
                    snapshot_words = [x for x in tokens if len(x) > 2]
                    snapshot_words = [stemmer.stem(x) for x in snapshot_words if not x in stop_words]
                    words.extend(snapshot_words)

                if(len(words) > 0):
                    task_words_ungrouped.append(words)
                    task_order.append(row["task"])
        
        task_counters = [Counter(x) for x in task_words_ungrouped]
        if(not ungrouped):
            counters = defaultdict(Counter)
            for counter, task in zip(task_counters, task_order):
                counters[task] += counter
            task_order, task_counters = zip(*counters.items())
            
        if(using_tfidf):
            task_counters = tfidf(task_counters, weighted)

        for task in task_counters:
            norm = np.linalg.norm(list(task.values()))
            for word in task:
                task[word] = task[word]/norm

        return task_counters, task_order
    
    
def sim(v1, v2):
    return cosine_similarity(v1.reshape(1,-1), v2.reshape(1,-1))[0][0]

def create_results(method, predicted, expected, author, participant):
    return [{"method": method, "predicted": x, "expected": y, "author": author, "participant": participant, "correct": equals(x, y)} for x,y in zip(predicted, expected)]

prob = []

def predict_simple(tasks, order, phrases):
    predictions = []
    expected = []
    durations = []
    sim_vocab = []

    for task, actual in zip(tasks, order):
        expected.append(actual)

        scores = dict()
        cover_scores = dict()
        words = []
        cover = {}
        
        for _, row in phrases.iterrows():
            search_terms = word_tokenize(row["phrase"])
            search_terms = [x.lower() for x in search_terms if not x.lower() in stop_words]
            search_terms = [stemmer.stem(x) for x in search_terms]
            
            
            scores[row["expected"]] = tfidf_cosine_similarity(task, search_terms)

        pred = get_prediction(scores)
        predictions.append(pred)
    return predictions, expected



In [12]:
df = pd.read_excel("../phrases_medium_and_high_v2.xlsx")    
results = []
#participants = ["P01", "P02", "P03", "P04", "P05", "P06", "P07", "P08", "P11", "P12", "P13", "P14", "P15", "P16", "P17", "P18", "P19"]
participants = ["P01", "P02", "P03", "P04", "P05", "P06", "P14", "P15", "P16", "P17", "P18", "P19"]

results_tfidf = []
results_tf = []
results_rake = []
results_rake_tfidf = []

path_to_data = "../../archives/"
task_extractor = ScreenshotTaskExtractor()


for participant in participants:
    print("START ----- ")
    print(participant)
    
    
    #if(participant in cache):
    #    tasks,order = cache[participant]
    #else:
    #    tasks, order = task_extractor.get_tasks_for_participant(path_to_data, participant, word_tokenize, using_tfidf=True, ungrouped=True)
    #    cache[participant] = (tasks, order)
    
    
    tasks, order = task_extractor.get_tasks_for_participant(path_to_data, participant, word_tokenize, using_tfidf=True, ungrouped=True)

    for author in df["author"].unique():
        task_descriptions = df[df["author"] == author]
        #print("Matching phrases by author: " + author)
        predicted,expected = predict_simple(tasks, order, task_descriptions)
        r = create_results("tfidf", predicted, expected, author, participant)
        results.extend(r)
    
    '''
    tasks, order = task_extractor.get_tasks_for_participant(path_to_data, participant, word_tokenize, using_tfidf=False, ungrouped=True)

    for author in df["author"].unique():
        task_descriptions = df[df["author"] == author]
        #print("Matching phrases by author: " + author)
        predicted,expected = predict_simple(tasks, order, task_descriptions)
        r = create_results("tf", predicted, expected, author, participant)
        results.extend(r)
        
    tasks, order = task_extractor.get_tasks_for_participant(path_to_data, participant, rake, using_tfidf=False, ungrouped=True)

    for author in df["author"].unique():
        task_descriptions = df[df["author"] == author]
        #print("Matching phrases by author: " + author)
        predicted,expected = predict_simple(tasks, order, task_descriptions)
        r = create_results("rake", predicted, expected, author, participant)
        results.extend(r)
        
    tasks, order = task_extractor.get_tasks_for_participant(path_to_data, participant, rake, using_tfidf=True, ungrouped=True)

    for author in df["author"].unique():
        task_descriptions = df[df["author"] == author]
        #print("Matching phrases by author: " + author)
        predicted,expected = predict_simple(tasks, order, task_descriptions)
        r = create_results("rake-tfidf", predicted, expected, author, participant)
        results.extend(r)
    '''
    print("END -----")
    
y_true = [x["expected"] for x in results]
y_pred = [x["predicted"] for x in results]

print(accuracy_score(y_true, y_pred))
print(precision_recall_fscore_support(y_true, y_pred, labels=[1,2,3,4,5,6]))
print(matthews_corrcoef(y_true, y_pred))

    
    
    

START ----- 
P01
END -----
START ----- 
P02
END -----
START ----- 
P03
END -----
START ----- 
P04
END -----
START ----- 
P05
END -----
START ----- 
P06
END -----
START ----- 
P14
END -----
START ----- 
P15
END -----
START ----- 
P16
END -----
START ----- 
P17
END -----
START ----- 
P18
END -----
START ----- 
P19
END -----
0.5656185567010309
(array([0.80331325, 0.47603834, 0.49485683, 0.35250241, 0.69043221,
       0.68353414]), array([0.62900943, 0.61570248, 0.45177665, 0.59552846, 0.61887417,
       0.5126506 ]), array([0.70555556, 0.53693694, 0.47233647, 0.44286578, 0.65269775,
       0.5858864 ]), array([4240, 2420, 3940, 2460, 3020, 3320]))
0.48308607381390956


In [9]:
df = pd.DataFrame(results)

In [245]:
df.to_excel("matching_results.xlsx")